In [2]:
import openai

client = openai.OpenAI()
messages = []

In [3]:
def get_weather(city):
    return "아테네의 평균 기온은 33도 입니다"

FUNCTION_MAP = {
    "get_weather": get_weather
    }


TOOLS = [
    {
        'type': "function",
        'function': {
            'name': 'get_weather',
            'description': 'A function to get the weather of a city.',
            'parameters': {
                'type': 'object',
                'required': ['city'],
                'properties': {
                    'city': {
                        'type': 'string',
                        'description': 'A name of city to get the weather of'
                    }
                }
            }
        }
    }
]

In [4]:
from openai.types.chat import ChatCompletionMessage
import json

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append(
            {
                'role': 'assistant',
                'content': message.content or "",
                'tool_calls': [
                   {
                    'id': tool_call.id,
                    'type': 'function',
                    'function': {
                        'name': tool_call.function.name,
                        'arguments': tool_call.function.arguments
                    }
                   } for tool_call in message.tool_calls 
                ]
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments={}

            function_to_run = FUNCTION_MAP.get(function_name)
            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args{arguments} for a result{result}")

            messages.append(
                {
                    'role': 'tool',
                    'content': result,
                    'tool_call_id': tool_call.id,
                    'name': function_name
                }
            )
        call_ai() # tool_calls가 사라질 때 까지
        
    else: # else로 나왔다는 건 tool_calls가 없다는 의미
        messages.append({'role': 'assistant', 'content': message.content})
        print(f"AI: {message.content}")

def call_ai():
    response = client.chat.completions.create(
        model='gpt-4.1-mini',
        messages=messages,
        tools=TOOLS
    )
    process_ai_response(response.choices[0].message)

In [5]:
while True:
    message = input("llm에게 질문해보세요")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({'role': 'user', 'content': message})
        print('User:', message)
        call_ai()

User: 내 이름은 정재현이야
AI: 안녕하세요, 정재현님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?
User: 내 이름이 뭐야?
AI: 당신의 이름은 정재현입니다. 다른 도움이 필요하신가요?
User: 아테네의 평균 기온이 궁금해
calling function: get_weather with {"city":"Athens"}
Ran get_weather with args{'city': 'Athens'} for a result아테네의 평균 기온은 33도 입니다
AI: 아테네의 평균 기온은 33도입니다. 더 궁금한 점이 있나요?
User: 코딩을 잘하는 방법을 알려줘
AI: 코딩을 잘하는 방법에는 여러 가지가 있지만, 몇 가지 중요한 팁을 알려드릴게요:

1. 기본기를 탄탄히 하세요: 프로그래밍 언어의 문법과 기본 개념을 잘 이해하는 것이 중요합니다.
2. 꾸준히 연습하세요: 문제를 직접 풀어보면서 경험을 쌓는 것이 가장 효과적입니다.
3. 작은 프로젝트부터 시작하세요: 간단한 프로그램을 만들어보면서 실력을 키워나가세요.
4. 코드를 많이 읽어보세요: 다른 사람이 작성한 코드를 이해하고 분석하는 것도 좋은 공부 방법입니다.
5. 디버깅 능력을 키우세요: 오류를 찾고 해결하는 능력은 매우 중요합니다.
6. 최신 기술과 트렌드를 따라가세요: 꾸준히 공부하고 새로운 것을 배우는 자세가 필요합니다.
7. 커뮤니티에 참여하세요: 질문하고 답변하면서 배울 수 있는 기회가 많습니다.

필요하다면 특정 언어나 분야에 맞춘 공부 방법도 알려드릴 수 있습니다. 어떤 부분이 더 궁금한가요?


In [6]:
messages

[{'role': 'user', 'content': '내 이름은 정재현이야'},
 {'role': 'assistant', 'content': '안녕하세요, 정재현님! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?'},
 {'role': 'user', 'content': '내 이름이 뭐야?'},
 {'role': 'assistant', 'content': '당신의 이름은 정재현입니다. 다른 도움이 필요하신가요?'},
 {'role': 'user', 'content': '아테네의 평균 기온이 궁금해'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_jy3RG2erMo2hLcBfi4CScvb2',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Athens"}'}}]},
 {'role': 'tool',
  'content': '아테네의 평균 기온은 33도 입니다',
  'tool_call_id': 'call_jy3RG2erMo2hLcBfi4CScvb2',
  'name': 'get_weather'},
 {'role': 'assistant', 'content': '아테네의 평균 기온은 33도입니다. 더 궁금한 점이 있나요?'},
 {'role': 'user', 'content': '코딩을 잘하는 방법을 알려줘'},
 {'role': 'assistant',
  'content': '코딩을 잘하는 방법에는 여러 가지가 있지만, 몇 가지 중요한 팁을 알려드릴게요:\n\n1. 기본기를 탄탄히 하세요: 프로그래밍 언어의 문법과 기본 개념을 잘 이해하는 것이 중요합니다.\n2. 꾸준히 연습하세요: 문제를 직접 풀어보면서 경험을 쌓는 것이 가장 효과적입니다.\n3. 작은 프로젝트부터 시작하세요: 간단한 프로그램을 만들어보면서 실력을 키워나가세요.\n4. 코드를 많이 읽어보세요: 다른 사람이 작성